In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 6.4 MB 25.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Importing Dependecies

In [ ]:
# Importing the necessary libraries

import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
from google.colab import files

In [ ]:
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

In [ ]:
data = yf.download("RELIANCE.NS",
                   start=start_date,
                   end=end_date,
                   progress=False)

In [ ]:
data["Date"] = data.index
print(data["Date"].head())

Date
2008-09-16   2008-09-16
2008-09-17   2008-09-17
2008-09-18   2008-09-18
2008-09-19   2008-09-19
2008-09-22   2008-09-22
Name: Date, dtype: datetime64[ns]


In [ ]:
data = data[["Date", "Open", "High", "Low", "Close",
             "Adj Close", "Volume"]]

In [ ]:
data.reset_index(drop=True, inplace=True)
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3366,2022-05-19,2484.0,2512.500000,2466.050049,2479.850098,2479.850098,6783641
3367,2022-05-20,2512.5,2644.399902,2502.000000,2624.449951,2624.449951,10057508
3368,2022-05-23,2631.5,2653.550049,2593.000000,2606.899902,2606.899902,7039144
3369,2022-05-24,2607.0,2638.000000,2597.699951,2615.850098,2615.850098,6243086
3370,2022-05-25,2635.0,2656.699951,2601.500000,2612.000000,2612.000000,5867760


In [ ]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],
                                        high=data["High"],
                                        low=data["Low"],
                                        close=data["Close"])])
figure.update_layout(title="Reliance Stock Price Analysis",
                     xaxis_rangeslider_visible=True)
figure.show()

In [ ]:
# Checking the Correlation

correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    0.999892
High         0.999843
Low          0.999826
Open         0.999585
Volume       0.044182
Name: Close, dtype: float64


## Training LSTM for Stock Price Prediciton

In [ ]:
# Selecting the Input and Output features

X = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
X = X.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

In [ ]:
# Splitting the Training and Testing data

from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

In [ ]:
# Neural Network architecture for LSTM

from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(Xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training our neural network for model for Stock price prediction

model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(Xtrain,
          ytrain,
          batch_size=64,
          epochs=50)

Epoch 1/50
43/43 [==============================] - 4s 14ms/step - loss: 1142800.5000
Epoch 2/50
43/43 [==============================] - 0s 11ms/step - loss: 1085338.8750
Epoch 3/50
43/43 [==============================] - 0s 11ms/step - loss: 1015917.3125
Epoch 4/50
43/43 [==============================] - 0s 11ms/step - loss: 934337.0625
Epoch 5/50
43/43 [==============================] - 0s 11ms/step - loss: 845578.3750
Epoch 6/50
43/43 [==============================] - 0s 11ms/step - loss: 755650.6875
Epoch 7/50
43/43 [==============================] - 0s 11ms/step - loss: 669349.5625
Epoch 8/50
43/43 [==============================] - 1s 20ms/step - loss: 594083.2500
Epoch 9/50
43/43 [==============================] - 1s 21ms/step - loss: 532925.4375
Epoch 10/50
43/43 [==============================] - 1s 20ms/step - loss: 485644.0000
Epoch 11/50
43/43 [==============================] - 1s 21ms/step - loss: 453798.5625
Epoch 12/50
43/43 [==============================] - 1s 21ms

In [ ]:
# Testing this model by giving input values 

import numpy as np
#features = [Open, High, Low, Volume]
features = np.array([[177.089996, 180.419998, 177.070007, 74919600]])
model.predict(features)

array([[167.60043]], dtype=float32)

## Pickle

In [ ]:
import pickle

model = pickle.dump(model, open("Model_Reliance_LSTM.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://8dec8daa-7aba-4b91-a3a3-89e95abd197d/assets


INFO:tensorflow:Assets written to: ram://8dec8daa-7aba-4b91-a3a3-89e95abd197d/assets


In [ ]:
model = pickle.load(open("Model_Reliance_LSTM.pkl", "rb"))

In [18]:
files.download("Model_Reliance_LSTM.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>